In [17]:
from pathlib import Path
import sys
import os
import json
from pybloom import BloomFilter
import math, time
import pandas as pd
import regex as re

In [18]:
queries = pd.read_csv("queries.csv")
queries['files'].value_counts()

LaptopPeriferals       2500
Laptop                 2500
Mobile                 2500
Refrigerator           2500
Tablet                 2500
TV                     2500
WashingMachine         2500
WearableSmartDevice    2500
CameraLens             2157
Name: files, dtype: int64

In [19]:
queries.head(20)

,queries,files
0,minimum-focusing-distance 0.49-m,CameraLens
1,maximum-magnification-(x),CameraLens
2,maximum-focal-length,CameraLens
3,lens-cap snap-on,CameraLens
4,black,CameraLens
5,model f013n,CameraLens
6,warranty 4,CameraLens
7,aperture-with-max-focal-length f/3.5-5.6g,CameraLens
8,water-resistant,CameraLens
9,diameter 74.4-mm,CameraLens


In [86]:
# query = ["Rectangle Unisex Black DAYNEO"]
# query = queries['queries'][14571]
query = 'filter-attachment-size 58-mm maximum-focal-length 250-mm'

In [85]:
queries['files'][14571]

'Tablet'

In [70]:
query.split()

['battery-features',
 'polymer-ion-battery',
 'video-formats-supported',
 'mp4,-xvid,-flv,-mpeg,-mkv,-wmv',
 'touchscreen-type',
 'no',
 'model-number',
 'slide-3g-6095-q700',
 'other-connectivity-features',
 'bluetooth,-wi-fi,-wi-fi-display']

In [21]:
initial_data = []
path = sys.path[0] + '\\data'
json_files = [pos_json for pos_json in os.listdir(
        path) if pos_json.endswith('.json')]
for index, js in enumerate(json_files):
    with open(os.path.join(path, js)) as json_file:
        data = json.load(json_file)
        initial_data.append(data)

In [22]:
temp1 = [
            "5326",
            "5329",
            "5332",
            "5333",
            "5335",
            "5336",
            "5338",
            "5339",
            "5340",
            "5341",
            "5344",
            "5348",
            "5353",
            "5355",
            "5356",
            "5357",
            "5359",
            "5359",
            "5361",
            "5362",
            "5363",
            "5367",
            "5369",
            "5373",
            "5375",
            "5379",
            "5383",
            "5384",
            "5385",
            "5386",
            "5387",
            "5392",
            "5393",
            "5395",
            "5396",
            "5397",
            "5399",
            "5405",
            "5409",
            "5410",
            "5412",
            "5414",
            "5417",
            "5421",
            "5425",
            "5428",
            "5429",
            "5431",
            "5442",
            "5443",
            "5447",
            "5449",
            "5451",
            "5455",
            "5458",
            "5459",
            "5460",
            "5466",
            "5467",
            "5469",
            "5473",
            "5478",
            "5480",
            "5481",
            "5484",
            "5485",
            "5489",
            "5490",
            "5494",
            "5497",
            "5499",
            "5509",
            "5511",
            "5513",
            "5515",
            "5518",
            "5521",
            "5522",
            "5524",
            "5526",
            "5527",
            "5528",
            "5531",
            "5532",
            "5534",
            "5536",
            "5543",
            "5544",
            "5545",
            "5547",
            "5551",
            "5553",
            "5554",
            "5556",
            "5566",
            "5568",
            "5571",
            "5573",
            "5574",
            "5580",
            "5583",
            "5584",
            "5586",
            "5588",
            "5591",
            "5593",
            "5594",
            "5599",
            "5601",
            "5611",
            "5614",
            "5616",
            "5617",
            "5620",
            "5621",
            "5628",
            "5630",
            "5631",
            "5632",
            "5641",
            "5644",
            "5651",
            "5652",
            "5654",
            "5656",
            "5657",
            "5658",
            "5662",
            "5664",
            "5670",
            "5675",
            "5676",
            "5677",
            "5680",
            "5681",
            "5684",
            "5686",
            "5691",
            "5693",
            "5694",
            "5698",
            "5701",
            "5704",
            "5718",
            "5718",
            "5719",
            "5727",
            "5731",
            "5735",
            "5738",
            "5741",
            "5744",
            "5746",
            "5747",
            "5749",
            "5755",
            "5756",
            "5760",
            "5761",
            "5762",
            "5764",
            "5766",
            "5769",
            "5771",
            "5773",
            "5776",
            "5777",
            "5779",
            "5782",
            "5783"
      ]
temp2 = [
            "5326",
            "5329",
            "5332",
            "5335",
            "5338",
            "5339",
            "5341",
            "5344",
            "5350",
            "5351",
            "5352",
            "5362",
            "5369",
            "5372",
            "5373",
            "5375",
            "5387",
            "5389",
            "5390",
            "5395",
            "5402",
            "5403",
            "5409",
            "5414",
            "5416",
            "5418",
            "5419",
            "5421",
            "5422",
            "5428",
            "5431",
            "5432",
            "5433",
            "5434",
            "5438",
            "5446",
            "5452",
            "5456",
            "5458",
            "5459",
            "5478",
            "5482",
            "5484",
            "5486",
            "5488",
            "5489",
            "5490",
            "5497",
            "5499",
            "5501",
            "5504",
            "5508",
            "5518",
            "5522",
            "5527",
            "5528",
            "5532",
            "5533",
            "5540",
            "5554",
            "5560",
            "5568",
            "5569",
            "5573",
            "5584",
            "5589",
            "5591",
            "5594",
            "5598",
            "5599",
            "5601",
            "5625",
            "5628",
            "5635",
            "5643",
            "5654",
            "5658",
            "5660",
            "5661",
            "5662",
            "5674",
            "5675",
            "5684",
            "5693",
            "5696",
            "5701",
            "5702",
            "5727",
            "5739",
            "5740",
            "5743",
            "5747",
            "5749",
            "5755",
            "5756",
            "5758",
            "5760",
            "5764",
            "5769",
            "5775",
            "5781"
      ]
temp3 = [
            "5325",
            "5325",
            "5326",
            "5326",
            "5328",
            "5329",
            "5329",
            "5331",
            "5332",
            "5335",
            "5335",
            "5338",
            "5343",
            "5344",
            "5345",
            "5345",
            "5345",
            "5346",
            "5346",
            "5348",
            "5352",
            "5353",
            "5353",
            "5354",
            "5355",
            "5355",
            "5359",
            "5359",
            "5360",
            "5361",
            "5362",
            "5366",
            "5367",
            "5367",
            "5370",
            "5371",
            "5371",
            "5371",
            "5372",
            "5373",
            "5375",
            "5375",
            "5377",
            "5378",
            "5379",
            "5381",
            "5383",
            "5383",
            "5385",
            "5390",
            "5390",
            "5391",
            "5392",
            "5392",
            "5393",
            "5395",
            "5396",
            "5396",
            "5401",
            "5401",
            "5402",
            "5402",
            "5403",
            "5404",
            "5404",
            "5406",
            "5407",
            "5411",
            "5414",
            "5416",
            "5416",
            "5416",
            "5417",
            "5417",
            "5418",
            "5418",
            "5419",
            "5419",
            "5420",
            "5420",
            "5422",
            "5423",
            "5424",
            "5424",
            "5425",
            "5427",
            "5427",
            "5428",
            "5429",
            "5431",
            "5432",
            "5432",
            "5432",
            "5433",
            "5433",
            "5434",
            "5434",
            "5435",
            "5435",
            "5436",
            "5436",
            "5438",
            "5439",
            "5440",
            "5440",
            "5441",
            "5441",
            "5442",
            "5445",
            "5445",
            "5446",
            "5447",
            "5450",
            "5450",
            "5452",
            "5453",
            "5453",
            "5454",
            "5455",
            "5455",
            "5456",
            "5457",
            "5459",
            "5460",
            "5460",
            "5462",
            "5463",
            "5464",
            "5465",
            "5465",
            "5466",
            "5469",
            "5470",
            "5471",
            "5473",
            "5473",
            "5475",
            "5476",
            "5477",
            "5477",
            "5478",
            "5481",
            "5482",
            "5482",
            "5483",
            "5483",
            "5484",
            "5485",
            "5485",
            "5492",
            "5493",
            "5494",
            "5494",
            "5496",
            "5496",
            "5497",
            "5498",
            "5501",
            "5501",
            "5502",
            "5502",
            "5503",
            "5504",
            "5504",
            "5505",
            "5506",
            "5511",
            "5511",
            "5512",
            "5513",
            "5513",
            "5514",
            "5514",
            "5515",
            "5517",
            "5517",
            "5521",
            "5521",
            "5522",
            "5522",
            "5524",
            "5525",
            "5526",
            "5527",
            "5530",
            "5532",
            "5533",
            "5535",
            "5538",
            "5538",
            "5539",
            "5539",
            "5542",
            "5542",
            "5543",
            "5545",
            "5548",
            "5550",
            "5551",
            "5551",
            "5556",
            "5556",
            "5559",
            "5559",
            "5561",
            "5562",
            "5563",
            "5564",
            "5564",
            "5566",
            "5568",
            "5570",
            "5570",
            "5573",
            "5573",
            "5577",
            "5577",
            "5578",
            "5578",
            "5579",
            "5582",
            "5583",
            "5583",
            "5584",
            "5584",
            "5587",
            "5588",
            "5588",
            "5589",
            "5592",
            "5592",
            "5593",
            "5594",
            "5596",
            "5596",
            "5598",
            "5599",
            "5601",
            "5606",
            "5606",
            "5610",
            "5613",
            "5616",
            "5616",
            "5620",
            "5620",
            "5621",
            "5622",
            "5622",
            "5625",
            "5626",
            "5626",
            "5628",
            "5633",
            "5633",
            "5634",
            "5634",
            "5635",
            "5635",
            "5636",
            "5637",
            "5637",
            "5638",
            "5642",
            "5643",
            "5646",
            "5648",
            "5649",
            "5649",
            "5651",
            "5652",
            "5652",
            "5654",
            "5654",
            "5656",
            "5656",
            "5657",
            "5661",
            "5662",
            "5665",
            "5667",
            "5670",
            "5670",
            "5671",
            "5671",
            "5672",
            "5675",
            "5679",
            "5680",
            "5680",
            "5683",
            "5683",
            "5683",
            "5685",
            "5689",
            "5689",
            "5690",
            "5690",
            "5691",
            "5691",
            "5693",
            "5696",
            "5698",
            "5700",
            "5703",
            "5703",
            "5706",
            "5706",
            "5707",
            "5710",
            "5711",
            "5711",
            "5712",
            "5712",
            "5713",
            "5714",
            "5716",
            "5716",
            "5717",
            "5717",
            "5721",
            "5721",
            "5722",
            "5724",
            "5727",
            "5729",
            "5729",
            "5730",
            "5734",
            "5734",
            "5735",
            "5737",
            "5738",
            "5740",
            "5740",
            "5741",
            "5746",
            "5752",
            "5752",
            "5754",
            "5754",
            "5755",
            "5756",
            "5757",
            "5759",
            "5761",
            "5762",
            "5764",
            "5764",
            "5766",
            "5768",
            "5768",
            "5769",
            "5771",
            "5771",
            "5773",
            "5773",
            "5773",
            "5775",
            "5775",
            "5779",
            "5780",
            "5780",
            "5781",
            "5781",
            "5782",
            "5782",
            "5783",
            "5784"
      ]
temp4 = [
            "5325",
            "5367",
            "5436",
            "5502",
            "5538",
            "5573",
            "5634",
            "5667",
            "5764"
      ]

result = list(set(temp1) & set (temp2) & set (temp3) & set (temp4))
print(result)

['5764', '5573']


In [104]:
keywords = query.split()

itemSet = []

items = []

start = time.time()

for i in range(len(initial_data)):
    data_filter = initial_data[i]
    for j in range(len(data_filter)):
        count = 0
        for k in range(len(keywords)):
            keys = list(map(lambda x: x.lower(), data_filter[j].keys()))
            values = list(map(lambda x: x.lower(), data_filter[j].values()))
            if keywords[k] in keys or keywords[k] in values:
                count = count + 1
            else:
                break
        if count == len(keywords):
            itemSet.append(j)
    if len(itemSet) != 0:
        for item in range(len(itemSet)):
            items.append(initial_data[i][itemSet[item]]) 
        break
    itemSet = []
    
end = time.time()
print("{:5.6f} seconds to retreive the itemSet".format(end - start))
        
                
# print(items)
if len(itemSet) == 0:
    print("No common items found in the itemSet")
else:
    print(itemSet)
            

0.001492 seconds to retreive the itemSet
[0]


In [79]:
keywords

['battery-features',
 'polymer-ion-battery',
 'video-formats-supported',
 'mp4,-xvid,-flv,-mpeg,-mkv,-wmv',
 'touchscreen-type',
 'no',
 'model-number',
 'slide-3g-6095-q700',
 'other-connectivity-features',
 'bluetooth,-wi-fi,-wi-fi-display']

In [45]:
library = {}
target_names = []

In [46]:
path = sys.path[0] + '\\data_with_ids'
json_files = [pos_json for pos_json in os.listdir(
        path) if pos_json.endswith('.json')]
for index, js in enumerate(json_files):
    with open(os.path.join(path, js)) as json_file:
        data = json.load(json_file)
        filename = Path(json_file.name)
        target_name = re.sub('.json','',filename.name)
        print(target_name)
        target_names.append(target_name)
        library[target_name] = {'data':{},'inverted_index_ds':{},'filter':{}}
        library[target_name]['data'] = data
        

CameraLens
Laptop
LaptopPeriferals
Mobile
Refrigerator
Tablet
TV
WashingMachine
WearableSmartDevice


In [47]:
# library['CameraLens']['inverted_index_ds']
# print(target_names)

['CameraLens', 'Laptop', 'LaptopPeriferals', 'Mobile', 'Refrigerator', 'Tablet', 'TV', 'WashingMachine', 'WearableSmartDevice']


In [48]:
i = 0
path = sys.path[0] + '\\inverted-index-ds'
json_files = [pos_json for pos_json in os.listdir(
        path) if pos_json.endswith('.json')]
for index, js in enumerate(json_files):
    with open(os.path.join(path, js)) as json_file:
        data = json.load(json_file)
        library[target_names[i]]['inverted-index-ds'] = data
        i = i + 1

In [49]:
library['CameraLens']['inverted-index-ds']

{'aperture-with-max-focal-length': {'f/4-5.6-is-ii': ['1001'],
  'f/1.8-g': ['1002'],
  'f/1.8-stm': ['1003'],
  'f/3.5-5.6g-ed-vr': ['1004'],
  'f/3.5-5.6g': ['1005'],
  'f/1.8': ['1007', '1008'],
  'f/3.5-6.3': ['1009'],
  'f/4-l-usm': ['1010']},
 'aperture-with-min-focal-length': {'f/22---32': ['1001'],
  'f/16': ['1002'],
  'f/3.5': ['1004'],
  'f/22': ['1005', '1006', '1010']},
 'canon': ['1001', '1003', '1010'],
 'black': ['1001',
  '1002',
  '1003',
  '1004',
  '1005',
  '1006',
  '1007',
  '1008',
  '1009',
  '1010'],
 'canon-dslrs': ['1001', '1003'],
 'dimensions': {'70-x-108-mm-(diameter-x-length)': ['1001'],
  '72-x-52.5-mm-(diameter-x-length)': ['1002'],
  '72-x-85-mm-(diameter-x-length)': ['1005'],
  '78.1-x-99-mm-(diameter-x-length)': ['1006']},
 'mm': ['1001'],
 'filter-attachment-size': {'58-mm': ['1001', '1002'],
  '49-mm': ['1003'],
  '67-mm': ['1004', '1005', '1006', '1007', '1009'],
  '77-mm': ['1010']},
 'canon-mount': ['1001', '1006', '1008', '1009'],
 'telephoto'

In [50]:
i = 0
path = sys.path[0] + '\\Bloom-Filters'
filter_files = [pos_json for pos_json in os.listdir(
        path)]
for index, js in enumerate(filter_files):
    with open(os.path.join(path, js),'rb') as filter_file:
        library[target_names[i]]['filter'] = BloomFilter.fromfile(filter_file)
        i = i + 1

In [51]:
library['CameraLens']['filter']

In [105]:
itemSet = []

final_items = []

index = 0

start = time.time()

keywords = query.split()

filenames = library.keys()

target_file = ""

for filename in filenames:
    count = 0
    while count < len(keywords):
        if keywords[count] in library[filename]['filter']:
            if(type(library[filename]['inverted-index-ds'][keywords[count]]) is dict):
                key_level0 = keywords[count]
                count = count + 1
                if keywords[count] in library[filename]['filter']:
                    itemSet.append(library[filename]['inverted-index-ds'][key_level0][keywords[count]])
                else:
                    keys = library[filename]['inverted-index-ds'][keywords[count]].keys()
                    for key in keys:
                        itemSet.append(library[filename]['inverted-index-ds'][keywords[count]][key])
            else:
                itemSet.append(library[filename]['inverted-index-ds'][keywords[count]])
        else: 
            break
        count = count + 1
        
    if count == len(keywords):
      items = list(set.intersection(*map(set,itemSet)))
      target_file = filename
      break
  
    itemSet = []
    
if len(itemSet) == 0:
    print("No items found in the data set")   
else:
    for item_id in items:
        final_items.append(library[filename]['data'][item_id])
        
end = time.time()
print("{:5.6f} seconds to retreive the itemSet".format(end - start))
    
# print(final_items)
# print(count)
print(items)

0.000000 seconds to retreive the itemSet
['1001']


In [94]:
keywords

['filter-attachment-size', '58-mm', 'maximum-focal-length', '250-mm']

In [75]:
keywords = query[0].split()
print(keywords)

['b']


In [77]:
query

'battery-features polymer-ion-battery video-formats-supported mp4,-xvid,-flv,-mpeg,-mkv,-wmv touchscreen-type no model-number slide-3g-6095-q700 other-connectivity-features bluetooth,-wi-fi,-wi-fi-display'

In [63]:
library[target_names[8]]['inverted-index-ds'][keywords[count]] is dict

False

In [74]:
keywords = [keyword.lower() for keyword in query[0].split()]
len(keywords)

1

In [106]:

# for i in range(9):
#     count = 0
#     keys = inverted_index_ds[i].keys();
#     for key in keys:
#         if(type(inverted_index_ds[i][key]) is dict):
#             keys_level1 = inverted_index_ds[i][key].keys();
#             count = count + len(keys_level1) + 1
#         else:
#             count = count + 1
#     print(str(count) + "\n")
            

In [5]:
from itertools import combinations
final_queries = []
query_keywords = ["Hey","Here","How are you","There","Where","SEE"]
for i in range(1,6):
    [final_queries.append(" ".join(keyword)) for keyword in combinations((query_keywords),i)]
print(final_queries)

['Hey', 'Here', 'How are you', 'There', 'Where', 'SEE', 'Hey Here', 'Hey How are you', 'Hey There', 'Hey Where', 'Hey SEE', 'Here How are you', 'Here There', 'Here Where', 'Here SEE', 'How are you There', 'How are you Where', 'How are you SEE', 'There Where', 'There SEE', 'Where SEE', 'Hey Here How are you', 'Hey Here There', 'Hey Here Where', 'Hey Here SEE', 'Hey How are you There', 'Hey How are you Where', 'Hey How are you SEE', 'Hey There Where', 'Hey There SEE', 'Hey Where SEE', 'Here How are you There', 'Here How are you Where', 'Here How are you SEE', 'Here There Where', 'Here There SEE', 'Here Where SEE', 'How are you There Where', 'How are you There SEE', 'How are you Where SEE', 'There Where SEE', 'Hey Here How are you There', 'Hey Here How are you Where', 'Hey Here How are you SEE', 'Hey Here There Where', 'Hey Here There SEE', 'Hey Here Where SEE', 'Hey How are you There Where', 'Hey How are you There SEE', 'Hey How are you Where SEE', 'Hey There Where SEE', 'Here How are yo

In [1]:
# temp = initial_data[0]
# check = 'Canon'
# print(check in temp[0].keys())